In [1]:
from DemoServerDB import ConnectDatabase

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7f429af14910; dsn: 'dbname=postgres_db user=postgres_user password=xxxxx host=developer.tuple-mia.com port=5432 sslmode=require', closed: 0>

In [4]:
query = 'SELECT * from master_table_history'

In [5]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

In [6]:
temp.cur.execute(query)

In [7]:
col_names = [desc[0] for desc in temp.cur.description]

In [8]:
master_table_his = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [9]:
max_date = master_table_his['custom_date_identifier'].max()

In [10]:
list_of_indexes_new = np.where(master_table_his['custom_date_identifier'] == max_date)

In [11]:
new_data = master_table_his.ix[list_of_indexes_new]

In [12]:
new_data.reset_index(drop=True, inplace=True)

In [13]:
new_data.head()

,id,cust_id,churn,engagement,percent_churn,cltv,value,percent_cltv,high_convertor,bio,lifestage,custom_date_identifier
0,702,6358882,None,None,None,None,None,None,High,7,None,2017-04-10 22:00:01.314963
1,1424,6292710,None,None,None,None,None,None,High,0,None,2017-04-10 22:00:01.314963
2,2172,6247369,None,None,None,None,None,None,High,2,None,2017-04-10 22:00:01.314963
3,2294,6181505,None,None,None,None,None,None,High,4,None,2017-04-10 22:00:01.314963
4,2466,6325984,0,Medium,0.465975372650681,0,Very Low,0,Low,4,Reactivated,2017-04-10 22:00:01.314963


In [14]:
list_of_indexes_old = np.where(master_table_his['custom_date_identifier'] != max_date)

In [15]:
old_data = master_table_his.ix[list_of_indexes_old]

In [16]:
old_data.head()

,id,cust_id,churn,engagement,percent_churn,cltv,value,percent_cltv,high_convertor,bio,lifestage,custom_date_identifier
0,4764,6151721,None,None,None,None,None,None,High,0,None,2017-04-04 22:00:01.209927
1,8,6236121,None,None,None,None,None,None,Low,0,None,2017-04-04 22:00:01.209927
2,6035,6330211,None,None,None,None,None,None,Low,5,None,2017-04-04 22:00:01.209927
3,3767,6348036,None,None,None,None,None,None,Low,0,None,2017-04-04 22:00:01.209927
4,4098,6148833,None,None,None,None,None,None,Low,4,None,2017-04-04 22:00:01.209927


In [17]:
cltv_value_old = old_data[['cltv', 'custom_date_identifier']]

In [18]:
cltv_value_old.head()

,cltv,custom_date_identifier
0,None,2017-04-04 22:00:01.209927
1,None,2017-04-04 22:00:01.209927
2,None,2017-04-04 22:00:01.209927
3,None,2017-04-04 22:00:01.209927
4,None,2017-04-04 22:00:01.209927


In [19]:
old_cltv_sum = cltv_value_old.groupby('custom_date_identifier').sum().reset_index()

In [20]:
old_cltv_sum

,custom_date_identifier,cltv
0,2017-04-04 22:00:01.209927,26816.3279244758916
1,2017-04-05 22:00:01.278206,NaN
2,2017-04-06 22:00:01.169862,NaN
3,2017-04-07 22:00:01.551212,0


In [21]:
old_cltv_count = cltv_value_old.groupby('custom_date_identifier').count().reset_index()

In [22]:
old_cltv_count

,custom_date_identifier,cltv
0,2017-04-04 22:00:01.209927,3087
1,2017-04-05 22:00:01.278206,0
2,2017-04-06 22:00:01.169862,0
3,2017-04-07 22:00:01.551212,1


In [23]:
old_cltv_avrg = old_cltv_sum.merge(old_cltv_count, on='custom_date_identifier')

In [24]:
old_cltv_avrg

,custom_date_identifier,cltv_x,cltv_y
0,2017-04-04 22:00:01.209927,26816.3279244758916,3087
1,2017-04-05 22:00:01.278206,NaN,0
2,2017-04-06 22:00:01.169862,NaN,0
3,2017-04-07 22:00:01.551212,0,1


In [25]:
old_cltv_avrg.dtypes

custom_date_identifier    datetime64[ns]
cltv_x                            object
cltv_y                             int64
dtype: object

In [26]:
old_cltv_avrg.cltv_x = old_cltv_avrg.cltv_x.astype('float64', raise_on_error = False)
old_cltv_avrg.cltv_y = old_cltv_avrg.cltv_y.astype('float64', raise_on_error = False)

In [27]:
old_cltv_avrg.dtypes

custom_date_identifier    datetime64[ns]
cltv_x                           float64
cltv_y                           float64
dtype: object

In [28]:
old_cltv_avrg['average_cltv'] = old_cltv_avrg.cltv_x / old_cltv_avrg.cltv_y.replace({ 0 : np.nan })

In [29]:
old_cltv_avrg

,custom_date_identifier,cltv_x,cltv_y,average_cltv
0,2017-04-04 22:00:01.209927,26816.327924,3087.0,8.686857
1,2017-04-05 22:00:01.278206,NaN,0.0,NaN
2,2017-04-06 22:00:01.169862,NaN,0.0,NaN
3,2017-04-07 22:00:01.551212,0.000000,1.0,0.000000


In [30]:
cltv_value_new = new_data[['cltv', 'custom_date_identifier']]

In [31]:
cltv_value_new

,cltv,custom_date_identifier
0,None,2017-04-10 22:00:01.314963
1,None,2017-04-10 22:00:01.314963
2,None,2017-04-10 22:00:01.314963
3,None,2017-04-10 22:00:01.314963
4,0,2017-04-10 22:00:01.314963
5,0,2017-04-10 22:00:01.314963
6,None,2017-04-10 22:00:01.314963
7,0,2017-04-10 22:00:01.314963
8,31.5437490360638,2017-04-10 22:00:01.314963


In [32]:
new_cltv_sum = cltv_value_new.groupby('custom_date_identifier').sum().reset_index()

In [33]:
new_cltv_count = cltv_value_new.groupby('custom_date_identifier').count().reset_index()

In [34]:
new_cltv_avrg = new_cltv_sum.merge(new_cltv_count, on='custom_date_identifier')

In [35]:
new_cltv_avrg.cltv_x = new_cltv_avrg.cltv_x.astype('float64', raise_on_error = False)
new_cltv_avrg.cltv_y = new_cltv_avrg.cltv_y.astype('float64', raise_on_error = False)

In [36]:
new_cltv_avrg['average_cltv'] = new_cltv_avrg.cltv_x / new_cltv_avrg.cltv_y.replace({ 0 : np.nan })

In [37]:
new_cltv_avrg

,custom_date_identifier,cltv_x,cltv_y,average_cltv
0,2017-04-10 22:00:01.314963,31.543749,4.0,7.885937


In [38]:
old_cltv_avrg

,custom_date_identifier,cltv_x,cltv_y,average_cltv
0,2017-04-04 22:00:01.209927,26816.327924,3087.0,8.686857
1,2017-04-05 22:00:01.278206,NaN,0.0,NaN
2,2017-04-06 22:00:01.169862,NaN,0.0,NaN
3,2017-04-07 22:00:01.551212,0.000000,1.0,0.000000


Average CLTV for last week is *** and increased/decreased by *** over previous week

In [40]:
Nutshell_1 = "Average CLTV for last week is {value1}$ and {value2} by {value3}$ over previous week".format(value1 = int(round(new_cltv_avrg.average_cltv.sum())), value2 = "increased" if int(round(new_cltv_avrg.average_cltv.sum())) > int(round(old_cltv_avrg.average_cltv.sum())) else "decreased", value3 = abs((int(round(new_cltv_avrg.average_cltv.sum())) - int(round(old_cltv_avrg.average_cltv.sum())))))

In [41]:
Nutshell_1

'Average CLTV for last week is 8$ and decreased by 1$ over previous week'

In [42]:
master_table_his.head()

,id,cust_id,churn,engagement,percent_churn,cltv,value,percent_cltv,high_convertor,bio,lifestage,custom_date_identifier
0,4764,6151721,None,None,None,None,None,None,High,0,None,2017-04-04 22:00:01.209927
1,8,6236121,None,None,None,None,None,None,Low,0,None,2017-04-04 22:00:01.209927
2,6035,6330211,None,None,None,None,None,None,Low,5,None,2017-04-04 22:00:01.209927
3,3767,6348036,None,None,None,None,None,None,Low,0,None,2017-04-04 22:00:01.209927
4,4098,6148833,None,None,None,None,None,None,Low,4,None,2017-04-04 22:00:01.209927


In [64]:
val_eng_current = master_table_his[['cust_id','engagement', 'value', 'custom_date_identifier']]

In [65]:
val_eng_current.head()

,cust_id,engagement,value,custom_date_identifier
0,6151721,None,None,2017-04-04 22:00:01.209927
1,6236121,None,None,2017-04-04 22:00:01.209927
2,6330211,None,None,2017-04-04 22:00:01.209927
3,6348036,None,None,2017-04-04 22:00:01.209927
4,6148833,None,None,2017-04-04 22:00:01.209927


In [66]:
xyz = val_eng_current.query('value == "Very Low"')

In [70]:
xyz.head()

,cust_id,engagement,value,custom_date_identifier
3986,2942914,Low,Very Low,2017-04-04 22:00:01.209927
3988,3129800,Low,Very Low,2017-04-04 22:00:01.209927
3989,3289765,Low,Very Low,2017-04-04 22:00:01.209927
3995,6148771,Low,Very Low,2017-04-04 22:00:01.209927
3996,6148794,Very Low,Very Low,2017-04-04 22:00:01.209927


In [69]:
xyz.query('engagement == "High"')

,cust_id,engagement,value,custom_date_identifier


In [73]:
value_old = old_data[['cust_id','custom_date_identifier','value']]

In [74]:
value_old.head()

,cust_id,custom_date_identifier,value
0,6151721,2017-04-04 22:00:01.209927,None
1,6236121,2017-04-04 22:00:01.209927,None
2,6330211,2017-04-04 22:00:01.209927,None
3,6348036,2017-04-04 22:00:01.209927,None
4,6148833,2017-04-04 22:00:01.209927,None


In [82]:
value_customers_old = value_old.groupby(['custom_date_identifier', 'cust_id','value']).size().reset_index()

In [84]:
value_customers_old_very_low = value_customers_old.query('value == "Very Low"')

In [86]:
value_customers_old_very_low.head()

,custom_date_identifier,cust_id,value,0
0,2017-04-04 22:00:01.209927,2942914,Very Low,1
2,2017-04-04 22:00:01.209927,3129800,Very Low,1
3,2017-04-04 22:00:01.209927,3289765,Very Low,1
9,2017-04-04 22:00:01.209927,6148771,Very Low,1
10,2017-04-04 22:00:01.209927,6148794,Very Low,1


*** CUSTOMERS saw Value Segment changed to Low to High yesterday which is X% of the total base

In [79]:
value_new = new_data[['cust_id','custom_date_identifier','value']]

In [80]:
value_customers_new = value_new.groupby(['custom_date_identifier', 'value']).size().reset_index()

In [81]:
value_customers_new

,custom_date_identifier,value,0
0,2017-04-10 22:00:01.314963,Very High,1
1,2017-04-10 22:00:01.314963,Very Low,3
